In [11]:
%pip install langchain langchain_openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import requests
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import json
from dotenv import load_dotenv
from langchain.globals import set_debug
from IPython.display import display, Markdown

load_dotenv()
set_debug(False)

In [2]:
DEPLOYMENT_NAME="gpt-35-turbo"
MODEL_NAME = "gpt-35-turbo"
API_VERSION="2024-02-15-preview"

RESULTS_PER_QUESTION = 3

SUMMARY_TEMPLATE = """{text} 
-----------
Using the above text, answer in short the following question: 
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""  # noqa: E501

SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)

SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 search queries to search online that form an "
            "objective opinion from the following: {question}\n"
            "You must respond with a list of strings in array: "
            '["query 1", "query 2", "query 3"].',
        ),
    ]
)

WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501

RESEARCH_REPORT_TEMPLATE = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""

RESEARCH_REPORT_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)

In [3]:
ddg_search = DuckDuckGoSearchAPIWrapper()

def web_search(query: str, num_results: int = RESULTS_PER_QUESTION):
    results = ddg_search.results(query, num_results)
    return [r["link"] for r in results]

def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME, 
    model_name=MODEL_NAME, 
    openai_api_version=API_VERSION,
    temperature=0)

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | llm | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

web_search_chain = RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()

search_question_chain = SEARCH_PROMPT | llm | StrOutputParser() | json.loads

full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

research_report_chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | RESEARCH_REPORT_PROMPT | llm | StrOutputParser()


In [8]:
result = research_report_chain.invoke({"question": "How to provision Azure resources by creating a github issue"})
display(Markdown(result))

# Report on Provisioning Azure Resources by Creating a GitHub Issue

## Introduction

Provisioning Azure resources using GitHub issues is a crucial aspect of modern software development workflows. GitHub issues provide a convenient way to track tasks, enhancements, and bugs for software projects. In the context of Azure resource provisioning, GitHub issues can be leveraged to automate the creation of Azure resources, such as resource groups, storage accounts, and machine learning workspaces. This report aims to explore the various methods and best practices for provisioning Azure resources by creating a GitHub issue, based on the information gathered from reputable sources.

## Methods for Provisioning Azure Resources via GitHub Issues

### Using Azure Login Action with OpenID Connect in GitHub Actions Workflow

One method for provisioning Azure resources using GitHub issues involves utilizing the Azure login action with OpenID Connect in a GitHub Actions workflow. This approach requires creating a Microsoft Entra application and service principal, assigning a role to the application, adding federated credentials, and creating GitHub secrets for the application's Client ID, Tenant ID, and Subscription ID. By following this method, developers can authenticate with Azure and interact with Azure resources using GitHub issues (Microsoft, n.d. a).

### Creating Workflows in GitHub Repository

Another approach to provisioning Azure resources through GitHub involves creating workflows in the repository to build, test, package, release, and deploy to Azure using GitHub Actions for Azure. These workflows can be set up to automate the provisioning of Azure services such as Azure App Service, Azure Functions, and Azure Key Vault. This method requires Azure and GitHub accounts, and developers can use a service principal or publish profile to connect to Azure from GitHub (Microsoft, n.d. b).

### Creating a Service Principal and Using Azure CLI Task in GitHub Actions

To provision Azure resources using GitHub, it is essential to create a service principal with a contributor role assigned at the subscription level. Subsequently, developers can use the Azure CLI task in GitHub actions to create the desired resources, such as a resource group and a Machine Learning workspace. It is crucial to add the necessary inline scripts in the GitHub workflow file to create the resources and configure the Azure ML workspace (Stack Overflow, n.d.).

### Leveraging Terraform and GitHub Actions with Remote Backend

A best practice for creating GitHub issues for Azure resource provisioning is to use Terraform to deploy infrastructure to Azure through GitHub Actions with a remote backend. This method allows for deploying infrastructure on Azure by running the appropriate workflow on GitHub, even without elevated permissions in Azure. Additionally, it ensures effective collaboration on the same infrastructure and reduces security risks by following the principle of least privilege (FPG Maas, n.d.).

### Step-by-Step Guide for Creating Azure Resources through GitHub

A comprehensive step-by-step guide for creating Azure resources through GitHub includes creating a service principal using the Azure CLI, storing the credentials as secrets in GitHub Actions, deploying the required resources for Terraform's backend, defining the infrastructure using Terraform, and providing a GitHub Actions workflow that deploys the infrastructure using Terraform when a release is published on the main branch (FPG Maas, n.d.).

## Best Practices for Provisioning Azure Resources via GitHub Issues

### Use of Terraform for Infrastructure Deployment

The use of Terraform for infrastructure deployment through GitHub Actions is highlighted as a best practice for provisioning Azure resources. By leveraging Terraform, developers can define and manage infrastructure as code, ensuring consistency and reproducibility in resource provisioning. Additionally, using Terraform with a remote backend allows for effective collaboration and reduces the need for elevated permissions in Azure, thereby enhancing security (FPG Maas, n.d.).

### Secure Management of Credentials

When provisioning Azure resources through GitHub issues, it is crucial to securely manage credentials and sensitive information. This includes storing credentials as secrets in GitHub Actions, ensuring that only authorized individuals have access to the necessary credentials for interacting with Azure resources. By following best practices for credential management, developers can mitigate the risk of unauthorized access and potential security breaches (Microsoft, n.d. a).

### Automation and Standardization

GitHub issues can be leveraged to automate the provisioning of Azure resources, leading to increased efficiency and standardization in the deployment process. By defining workflows and utilizing GitHub Actions, developers can ensure consistent and reliable provisioning of Azure resources across different environments and projects. This promotes a streamlined and scalable approach to resource provisioning, ultimately enhancing the overall development workflow (Microsoft, n.d. b).

## Conclusion

In conclusion, provisioning Azure resources by creating a GitHub issue involves various methods and best practices that aim to streamline the deployment process, enhance security, and promote collaboration. Leveraging GitHub Actions, service principals, Terraform, and secure credential management are key components of an effective approach to provisioning Azure resources through GitHub issues. By following best practices and utilizing the available tools and workflows, developers can ensure efficient and reliable provisioning of Azure resources while maintaining a high level of security and standardization.

## References

FPG Maas. (n.d.). Azure, Terraform & GitHub Actions. Retrieved from https://www.fpgmaas.com/blog/azure-terraform-github-actions

Microsoft. (n.d. a). Connect from Azure to GitHub. Retrieved from https://learn.microsoft.com/en-us/azure/developer/github/connect-from-azure

Microsoft. (n.d. b). GitHub Actions for Azure. Retrieved from https://learn.microsoft.com/en-us/azure/developer/github/github-actions

Stack Overflow. (n.d.). Creating Azure resource group or Azure ML workspace permission denied using GitHub Actions. Retrieved from https://stackoverflow.com/questions/75976105/creating-azure-resource-group-or-azure-ml-workspace-permission-denied-using-gi